# Numbers of features and empirical compounds by khipu preannotation

- Goal: data analysis to calculate number of empirical compounds in the khipu paper
- Citation: Li, S. and Zheng, S., 2023. Generalized tree structure to annotate untargeted metabolomics and stable isotope tracing data. Analytical chemistry, 95(15), pp.6212-6217. (https://pubs.acs.org/doi/10.1021/acs.analchem.2c05810)
- Original repo: https://github.com/shuzhao-li-lab/khipu

Three datasets: 
- ecoli_pos.tsv      
- yeast_pos_full.tsv
- yeast_neg.tsv (no 13C labeling)     

We will examine how many empirical compounds are in each dataset.

In [1]:
!pip install --upgrade khipu-metabolomics

Requirement already up-to-date: khipu-metabolomics in /opt/conda/lib/python3.7/site-packages (0.4.14)


In [2]:
from khipu.extended import *

In [3]:
for x in (adduct_search_patterns, isotope_search_patterns, extended_adducts):
    print(x, '\n')

[(21.982, 'Na/H'), (41.026549, 'ACN'), (35.9767, 'HCl'), (37.955882, 'K/H')] 

[(1.003355, '13C/12C', (0, 0.8)), (2.00671, '13C/12C*2', (0, 0.8)), (3.010065, '13C/12C*3', (0, 0.8)), (4.01342, '13C/12C*4', (0, 0.8)), (5.016775, '13C/12C*5', (0, 0.8)), (6.02013, '13C/12C*6', (0, 0.8)), (7.023485, '13C/12C*7', (0, 0.8)), (8.02684, '13C/12C*8', (0, 0.8)), (9.030195, '13C/12C*9', (0, 0.8)), (10.03355, '13C/12C*10', (0, 0.8)), (11.036905, '13C/12C*11', (0, 0.8)), (12.04026, '13C/12C*12', (0, 0.8))] 

[(1.0078, 'H'), (-1.0078, '-H'), (10.991, 'Na/H, double charged'), (0.5017, '13C/12C, double charged'), (117.02655, '-NH3'), (17.02655, 'NH3'), (-18.0106, '-H2O'), (18.0106, 'H2O'), (18.033823, 'NH4'), (27.01089904, 'HCN'), (37.94694, 'Ca/H2'), (32.026215, 'MeOH'), (43.96389, 'Na2/H2'), (67.987424, 'NaCOOH'), (83.961361, 'KCOOH'), (97.96737927, 'H2SO4'), (97.97689507, 'H3PO4')] 



## Part I. E. coli pos data


In [4]:
# 12C data
peaklist1 = read_features_from_text(open("ecoli_pos.tsv").read(),
                    id_col=0, mz_col=1, rtime_col=2, intensity_cols=(3, 6), delimiter="\t")

table header looks like:  ['id_number', 'mz', 'rtime', '12C_Ecoli_20220321_004', '12C_Ecoli_20220321_004_20220322095030', '12C_Ecoli_20220321_004_20220322130235']
Read 3602 feature lines


In [5]:
# filter out peaks of 0 intensity
peaklist1 = [p for p in peaklist1 if p['representative_intensity'] > 1]
print(len(peaklist1))

2973


In [6]:
# 13C data
peaklist2 = read_features_from_text(open("ecoli_pos.tsv").read(),
                    id_col=0, mz_col=1, rtime_col=2, intensity_cols=(6, 9), delimiter="\t")

table header looks like:  ['id_number', 'mz', 'rtime', '12C_Ecoli_20220321_004', '12C_Ecoli_20220321_004_20220322095030', '12C_Ecoli_20220321_004_20220322130235', '13C_Ecoli_20220321_004', '13C_Ecoli_20220321_004_20220322132355', '13C_Ecoli_20220321_004_20220322101150']
Read 3602 feature lines


In [7]:
# filter out peaks of 0 intensity
peaklist2 = [p for p in peaklist2 if p['representative_intensity'] > 1]
print(len(peaklist2))

3231


In [8]:
subnetworks, peak_dict, edge_dict = peaks_to_networks(peaklist1,
                    isotope_search_patterns,
                    adduct_search_patterns,
                    mz_tolerance_ppm=5,
                    rt_tolerance=2)

WV = Weavor(peak_dict, isotope_search_patterns=isotope_search_patterns, 
                adduct_search_patterns=adduct_search_patterns, 
                mz_tolerance_ppm=5, 
                mode='pos')

khipu_list = graphs_to_khipu_list(
        subnetworks, WV, mz_tolerance_ppm=5,)

print(len(subnetworks), len(khipu_list))

list_assigned_peaks = []
for KP in khipu_list:
    list_assigned_peaks += list(KP.feature_map.keys())
    
print(len(list_assigned_peaks))

Warning no match in isotope_pattern:  19.06400000000002
Unknown isotope match ~  (161.0962, 'F1913')
Warning no match in isotope_pattern:  27.090300000000013
Unknown isotope match ~  (169.1225, 'F1182')
Warning no match in isotope_pattern:  13.043700000000001
Unknown isotope match ~  (176.0708, 'F1657')
Warning no match in isotope_pattern:  15.050000000000011
Unknown isotope match ~  (291.1191, 'F3517')
352 352
860


In [9]:
ext_khipu_list, all_assigned_peaks = extend_khipu_list(khipu_list, peak_dict, 
                    extended_adducts, mz_tolerance_ppm=5,
                    rt_tolerance=2)

print(len(ext_khipu_list), len(all_assigned_peaks))

352 1096


**Got 352 empCpds from 12C data. Next to 13C data**

In [10]:
subnetworks, peak_dict, edge_dict = peaks_to_networks(peaklist2,
                    isotope_search_patterns,
                    adduct_search_patterns,
                    mz_tolerance_ppm=5,
                    rt_tolerance=2)

WV = Weavor(peak_dict, isotope_search_patterns=isotope_search_patterns, 
                adduct_search_patterns=adduct_search_patterns, 
                mz_tolerance_ppm=5, 
                mode='pos')

khipu_list = graphs_to_khipu_list(
        subnetworks, WV, mz_tolerance_ppm=5,)

print(len(subnetworks), len(khipu_list))

list_assigned_peaks = []
for KP in khipu_list:
    list_assigned_peaks += list(KP.feature_map.keys())
    
print(len(list_assigned_peaks))

Warning no match in isotope_pattern:  16.053999999999988
Unknown isotope match ~  (137.1188, 'F292')
Warning no match in isotope_pattern:  17.057299999999984
Unknown isotope match ~  (138.1221, 'F397')
Warning no match in isotope_pattern:  15.050899999999984
Unknown isotope match ~  (152.1378, 'F1251')
Warning no match in isotope_pattern:  16.05419999999998
Unknown isotope match ~  (153.1411, 'F1318')
Warning no match in isotope_pattern:  15.050899999999984
Unknown isotope match ~  (152.1378, 'F1252')
Warning no match in isotope_pattern:  16.05419999999998
Unknown isotope match ~  (153.1411, 'F1319')
Warning no match in isotope_pattern:  16.0539
Unknown isotope match ~  (123.1031, 'F570')
Warning no match in isotope_pattern:  13.0428
Unknown isotope match ~  (181.1392, 'F1967')
Warning no match in isotope_pattern:  13.043700000000001
Unknown isotope match ~  (176.0708, 'F1657')
397 397
1030


In [11]:
ext_khipu_list, all_assigned_peaks = extend_khipu_list(khipu_list, peak_dict, 
                    extended_adducts, mz_tolerance_ppm=5,
                    rt_tolerance=2)

print(len(ext_khipu_list), len(all_assigned_peaks))

397 1294


**Got 397 empCpds from 13C data.**

The minor difference from 12C data could be from culture media. 

## We will therefor examine combined 12C/13C numbers 

In [12]:
peaklist = read_features_from_text(open("ecoli_pos.tsv").read(),
                    id_col=0, mz_col=1, rtime_col=2, intensity_cols=(3, 9), delimiter="\t")

table header looks like:  ['id_number', 'mz', 'rtime', '12C_Ecoli_20220321_004', '12C_Ecoli_20220321_004_20220322095030', '12C_Ecoli_20220321_004_20220322130235', '13C_Ecoli_20220321_004', '13C_Ecoli_20220321_004_20220322132355', '13C_Ecoli_20220321_004_20220322101150']
Read 3602 feature lines


In [13]:
subnetworks, peak_dict, edge_dict = peaks_to_networks(peaklist,
                    isotope_search_patterns,
                    adduct_search_patterns,
                    mz_tolerance_ppm=5,
                    rt_tolerance=2)

WV = Weavor(peak_dict, isotope_search_patterns=isotope_search_patterns, 
                adduct_search_patterns=adduct_search_patterns, 
                mz_tolerance_ppm=5, 
                mode='pos')

khipu_list = graphs_to_khipu_list(
        subnetworks, WV, mz_tolerance_ppm=5,)

print(len(subnetworks), len(khipu_list))

list_assigned_peaks = []
for KP in khipu_list:
    list_assigned_peaks += list(KP.feature_map.keys())
    
print(len(list_assigned_peaks))

Warning no match in isotope_pattern:  16.053999999999988
Unknown isotope match ~  (137.1188, 'F292')
Warning no match in isotope_pattern:  17.057299999999984
Unknown isotope match ~  (138.1221, 'F397')
Warning no match in isotope_pattern:  15.050899999999984
Unknown isotope match ~  (152.1378, 'F1251')
Warning no match in isotope_pattern:  16.05419999999998
Unknown isotope match ~  (153.1411, 'F1318')
Warning no match in isotope_pattern:  15.050899999999984
Unknown isotope match ~  (152.1378, 'F1252')
Warning no match in isotope_pattern:  16.05419999999998
Unknown isotope match ~  (153.1411, 'F1319')
Warning no match in isotope_pattern:  16.0539
Unknown isotope match ~  (123.1031, 'F570')
Warning no match in isotope_pattern:  13.0428
Unknown isotope match ~  (181.1392, 'F1967')
Warning no match in isotope_pattern:  17.05710000000002
Unknown isotope match ~  (194.1596, 'F1548')
Warning no match in isotope_pattern:  13.043700000000001
Unknown isotope match ~  (176.0708, 'F1657')
Warning 

In [14]:
ext_khipu_list, all_assigned_peaks = extend_khipu_list(khipu_list, peak_dict, 
                    extended_adducts, mz_tolerance_ppm=5,
                    rt_tolerance=2)

print(len(ext_khipu_list), len(all_assigned_peaks))

548 1745


In [15]:
khipu_list[55].format_to_epds()

{'interim_id': 'root@F479',
 'neutral_formula_mass': 138.94452728323,
 'neutral_formula': None,
 'Database_referred': [],
 'identity': [],
 'MS1_pseudo_Spectra': [{'id': 'F479',
   'mz': 139.9515,
   'rtime': 66.69,
   'intensities': [1807302.0, 0.0, 0.0, 1866736.0, 1441162.0, 1857060.0],
   'representative_intensity': 1162043.3333333333,
   'parent_masstrack_id': '139.9515',
   'isotope': 'M0',
   'modification': 'M+H+',
   'ion_relation': 'M0,M+H+'},
  {'id': 'F2064',
   'mz': 163.9404,
   'rtime': 67.91,
   'intensities': [1369348.0, 0.0, 49750.0, 1398126.0, 391147.0, 651508.0],
   'representative_intensity': 643313.1666666666,
   'parent_masstrack_id': '163.9404',
   'isotope': '13C/12C*2',
   'modification': 'Na/H',
   'ion_relation': '13C/12C*2,Na/H'},
  {'id': 'F609',
   'mz': 141.9587,
   'rtime': 68.88,
   'intensities': [333247401.0,
    286675589.0,
    299087145.0,
    301648595.0,
    311680934.0,
    291801881.0],
   'representative_intensity': 304023590.8333333,
   'pare

In [16]:
has_13C = []
for KP in khipu_list:
    if len(list(KP.khipu_grid.index)) > 1:
        has_13C.append(KP.id)
        
print("Number of empCpds with more than 1 isotopes: ", len(has_13C))

Number of empCpds with more than 1 isotopes:  445


**Got 548 empCpds in E.coli pos data.**

Extended search annotated 1745 (1423 core) features.

Of 548 epmCpds, 
445 has 13C peaks; 548-445=103 has adducts only.

Singletons are 3602 - 1745 = 1857

**Almost identical when isotope search used up to 13/12C*12.**

(Got 548 khipus, with 1744 features )


## Part II. Yeast pos data

In [17]:
peaklist = read_features_from_text(open("yeast_pos_full.tsv").read(),
                    id_col=0, mz_col=1, rtime_col=2, intensity_cols=(6, 12), delimiter="\t")

table header looks like:  ['id_number', 'mz', 'rtime', 'cSelectivity', 'goodness_fitting', 'snr', 'posi-Yeast-12C14N-a', 'posi-Yeast-12C14N-b', 'posi-Yeast-12C14N-c', 'posi-Yeast-13C14N-a', 'posi-Yeast-13C14N-b', 'posi-Yeast-13C14N-c']
Read 14051 feature lines


In [18]:
subnetworks, peak_dict, edge_dict = peaks_to_networks(peaklist,
                    isotope_search_patterns,
                    adduct_search_patterns,
                    mz_tolerance_ppm=5,
                    rt_tolerance=2)

WV = Weavor(peak_dict, isotope_search_patterns=isotope_search_patterns, 
                adduct_search_patterns=adduct_search_patterns, 
                mz_tolerance_ppm=5, 
                mode='pos')

khipu_list = graphs_to_khipu_list(
        subnetworks, WV, mz_tolerance_ppm=5,)

print(len(subnetworks), len(khipu_list))

list_assigned_peaks = []
for KP in khipu_list:
    list_assigned_peaks += list(KP.feature_map.keys())
    
print(len(list_assigned_peaks))

Downsized input network with 2542 features, highest peak at F1005 
Warning no match in isotope_pattern:  76.06749999999997
Unknown isotope match ~  (269.1903, 'F515')
Warning no match in isotope_pattern:  92.0257
Unknown isotope match ~  (285.1485, 'F1644')
Warning no match in isotope_pattern:  92.06219999999999
Unknown isotope match ~  (285.185, 'F1684')
Warning no match in isotope_pattern:  94.0778
Unknown isotope match ~  (287.2006, 'F1856')
Warning no match in isotope_pattern:  103.13550000000001
Unknown isotope match ~  (296.2583, 'F2593')
Warning no match in isotope_pattern:  104.1199
Unknown isotope match ~  (297.2427, 'F2646')
Warning no match in isotope_pattern:  104.1389
Unknown isotope match ~  (297.2617, 'F2658')
Warning no match in isotope_pattern:  108.05709999999996
Unknown isotope match ~  (301.1799, 'F3031')
Warning no match in isotope_pattern:  110.0726
Unknown isotope match ~  (303.1954, 'F3227')
Warning no match in isotope_pattern:  112.08829999999998
Unknown isotop

/opt/conda/lib/python3.7/site-packages/scipy/optimize/minpack.py:795: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Warning no match in isotope_pattern:  19.065500000000014
Unknown isotope match ~  (272.2092, 'F790')
Warning no match in isotope_pattern:  21.072900000000033
Unknown isotope match ~  (274.2166, 'F917')
Warning no match in isotope_pattern:  27.0916
Unknown isotope match ~  (280.2353, 'F1293')
Warning no match in isotope_pattern:  16.05279999999999
Unknown isotope match ~  (197.1541, 'F102')
Warning no match in isotope_pattern:  19.063699999999983
Unknown isotope match ~  (200.165, 'F336')
Warning no match in isotope_pattern:  20.067099999999982
Unknown isotope match ~  (201.1684, 'F398')
Warning no match in isotope_pattern:  13.041800000000023
Unknown isotope match ~  (315.2533, 'F678')
Warning no match in isotope_pattern:  14.048100000000034
Unknown isotope match ~  (325.2488, 'F1528')
Warning no match in isotope_pattern:  16.052300000000002
Unknown isotope match ~  (209.1541, 'F922')
Warning no match in isotope_pattern:  17.0557
Unknown isotope match ~  (210.1575, 'F1042')
Warning no 

In [19]:
ext_khipu_list, all_assigned_peaks = extend_khipu_list(khipu_list, peak_dict, 
                    extended_adducts, mz_tolerance_ppm=5,
                    rt_tolerance=2)

print(len(ext_khipu_list), len(all_assigned_peaks))

1775 6310


**Search with more adducts**

In [20]:
more = '''7.985830404	H3BO3-3H2O
25.99639509	H3BO3-2H2O
37.94694	Ca-2H
39.99250931	NaOH
46.0054793	HCOOH
55.91969284	Ni-2H
55.96644655	KOH
60.02112937	C2H4O2
62.00039	H2CO3
62.9956429	HNO3
75.91176374	2K-2H
77.97732041	H2SiO3
82.003074	CH3COONa
83.98234	NaHCO3
84.97758753	NaNO3
95.9878851	H4SiO4
97.96807	CH2COOCa
97.97701124	CH3COOK
99.92525136	CrO3
99.95628	KHCO3
115.9408222	CH2COONi
119.9493239	NaHSO4
119.9588397	NaH2PO4
120.0242706	C3H8SiO3
135.9232611	KHSO4
135.9327769	KH2PO4
195.9537901	H3PO4 dimer'''.splitlines()
more = [x.split('\t') for x in more]
netID_additional_adducts = [
    (float(x[0]), x[1]) for x in more
]
print(netID_additional_adducts)

[(7.985830404, 'H3BO3-3H2O'), (25.99639509, 'H3BO3-2H2O'), (37.94694, 'Ca-2H'), (39.99250931, 'NaOH'), (46.0054793, 'HCOOH'), (55.91969284, 'Ni-2H'), (55.96644655, 'KOH'), (60.02112937, 'C2H4O2'), (62.00039, 'H2CO3'), (62.9956429, 'HNO3'), (75.91176374, '2K-2H'), (77.97732041, 'H2SiO3'), (82.003074, 'CH3COONa'), (83.98234, 'NaHCO3'), (84.97758753, 'NaNO3'), (95.9878851, 'H4SiO4'), (97.96807, 'CH2COOCa'), (97.97701124, 'CH3COOK'), (99.92525136, 'CrO3'), (99.95628, 'KHCO3'), (115.9408222, 'CH2COONi'), (119.9493239, 'NaHSO4'), (119.9588397, 'NaH2PO4'), (120.0242706, 'C3H8SiO3'), (135.9232611, 'KHSO4'), (135.9327769, 'KH2PO4'), (195.9537901, 'H3PO4 dimer')]


In [21]:
ext_khipu_list, all_assigned_peaks = extend_khipu_list(khipu_list, peak_dict, 
                    extended_adducts + netID_additional_adducts, 
                    mz_tolerance_ppm=5,
                    rt_tolerance=2)

print(len(ext_khipu_list), len(all_assigned_peaks))

1775 8049


In [22]:
has_13C = []
for KP in khipu_list:
    if len(list(KP.khipu_grid.index)) > 1:
        has_13C.append(KP.id)
        
print("Number of empCpds with more than 1 isotopes: ", len(has_13C))

Number of empCpds with more than 1 isotopes:  1582


**Got 1775 empCpds in yeast pos data**

Total features: 14051

core features: 4830

Extended search using khipu default adducts: 6310

Extended search using khipu default adducts plus NetID adducts: 8049

## Part III. Yeast neg data

In [23]:
peaklist = read_features_from_text(open("yeast_neg.tsv").read(),
                    id_col=0, mz_col=1, rtime_col=2, intensity_cols=(3, 6), delimiter="\t")

table header looks like:  ['id_number', 'mz', 'rtime', 'neg-12C14N-3-0ev', 'neg-12C14N-1-0ev', 'neg-12C14N-2-0ev']
Read 6286 feature lines


In [24]:
subnetworks, peak_dict, edge_dict = peaks_to_networks(peaklist,
                    isotope_search_patterns,
                    adduct_search_patterns_neg,
                    mz_tolerance_ppm=5,
                    rt_tolerance=2)

WV = Weavor(peak_dict, isotope_search_patterns=isotope_search_patterns, 
                adduct_search_patterns=adduct_search_patterns_neg, 
                mz_tolerance_ppm=5, 
                mode='neg')

khipu_list = graphs_to_khipu_list(
        subnetworks, WV, mz_tolerance_ppm=5,)

print(len(subnetworks), len(khipu_list))

list_assigned_peaks = []
for KP in khipu_list:
    list_assigned_peaks += list(KP.feature_map.keys())
    
print(len(list_assigned_peaks))

Warning no match in isotope_pattern:  0.0009000000000014552
Unknown isotope match ~  (157.1868, 'F9392')
Warning no match in isotope_pattern:  0.0011000000000080945
Unknown isotope match ~  (211.0833, 'F5242')
Warning no match in isotope_pattern:  0.0012000000000114142
Unknown isotope match ~  (227.066, 'F9696')
Warning no match in isotope_pattern:  13.042999999999978
Unknown isotope match ~  (266.1509, 'F5153')
Warning no match in isotope_pattern:  17.055199999999957
Unknown isotope match ~  (319.1004, 'F8323')
Warning no match in isotope_pattern:  13.043100000000038
Unknown isotope match ~  (338.2087, 'F10550')
908 908
2279


In [25]:
ext_khipu_list, all_assigned_peaks = extend_khipu_list(khipu_list, peak_dict, 
                    extended_adducts, mz_tolerance_ppm=5,
                    rt_tolerance=2)

print(len(ext_khipu_list), len(all_assigned_peaks))

908 2601


In [26]:
ext_khipu_list, all_assigned_peaks = extend_khipu_list(khipu_list, peak_dict, 
                    extended_adducts + netID_additional_adducts, 
                    mz_tolerance_ppm=5,
                    rt_tolerance=2)

print(len(ext_khipu_list), len(all_assigned_peaks))

908 2912


In [27]:
has_13C = []
for KP in khipu_list:
    if len(list(KP.khipu_grid.index)) > 1:
        has_13C.append(KP.id)
        
print("Number of empCpds with more than 1 isotopes: ", len(has_13C))

Number of empCpds with more than 1 isotopes:  763


**Got 908 empCpds in yeast neg data**

Total features: 6286

core features: 2279

Extended search using khipu default adducts: 2601

Extended search using khipu default adducts plus NetID adducts: 2912